In [2]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
import argparse
import math
from tensorflow.keras import optimizers
from tensorflow.keras import layers, initializers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from google.cloud import storage

In [1]:
from google.cloud import storage
client = storage.Client()
bucket = storage.Bucket("tfrecords_data")

In [2]:
bucket

<Bucket: None>

In [3]:
print(client.project)

drought-watch-297217


In [4]:
client.get_bucket('tfrecords_data')

<Bucket: tfrecords_data>

In [5]:
bucket = client.get_bucket('tfrecords_data')

In [6]:
bucket

<Bucket: tfrecords_data>

In [7]:
list(client.list_blobs(bucket))

[<Blob: tfrecords_data, data/.keep, 1606794361936555>,
 <Blob: tfrecords_data, data/packages/32b84a90e82f41bebde764ce0be9b8a369e9514bf2fd7c0090a2048e29a9af03/droughtwatch-1.0.tar.gz, 1606961340183751>,
 <Blob: tfrecords_data, data/train/_SUCCESS, 1606757932845610>,
 <Blob: tfrecords_data, data/train/packages/18a0afcaad5ebc0ee4c4b3ab8585a000861963227e202f29ccd14d497f929000/droughtwatch-1.0.tar.gz, 1606950250284447>,
 <Blob: tfrecords_data, data/train/packages/96a06f9e40ea9c8ed89e5329c05dd05515c3e0ea16f4e1d385b973590555ecf5/droughtwatch-1.0.tar.gz, 1606953999422902>,
 <Blob: tfrecords_data, data/train/packages/b55e09b3de7ecf9a6cf9821d20839f17547395b1fe3364aac2c2844624590127/droughtwatch-1.0.tar.gz, 1606954062123263>,
 <Blob: tfrecords_data, data/train/part-r-00000, 1606783796410444>,
 <Blob: tfrecords_data, data/train/part-r-00001, 1606795729376419>,
 <Blob: tfrecords_data, data/train/part-r-00002, 1606803419245464>,
 <Blob: tfrecords_data, data/train/part-r-00003, 1606803278454688>,
 <B

In [8]:
data0 = 'data/train/part-r-00000'

In [9]:
blob = bucket.get_blob(data0)

In [10]:
type(blob)

google.cloud.storage.blob.Blob

In [11]:
blob = bucket.blob(data0)
blobstring = blob.download_as_string()

In [12]:
x = blobstring.decode()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x87 in position 0: invalid start byte

In [14]:
blobbytes = blob.download_as_bytes()

In [16]:
with open("testfile", "wb") as file_obj:
    blob.download_to_file(file_obj)

In [17]:
def parse_tfrecords(filelist, batch_size, buffer_size, include_viz=False):
  # try a subset of possible bands
    def _parse_(serialized_example, keylist=['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']):
        example = tf.io.parse_single_example(serialized_example, features)

        def getband(example_key):
            img = tf.io.decode_raw(example_key, tf.uint8)
            return tf.reshape(img[:IMG_DIM**2], shape=(IMG_DIM, IMG_DIM, 1))

        bandlist = [getband(example[key]) for key in keylist]
        # combine bands into tensor
        image = tf.concat(bandlist, -1)

        # one-hot encode ground truth labels
        label = tf.cast(example['label'], tf.int32)
        label = tf.one_hot(label, NUM_CLASSES)

        return {'image': image}, label

    tfrecord_dataset = tf.data.TFRecordDataset(filelist)
    tfrecord_dataset = tfrecord_dataset.map(lambda x:_parse_(x)).shuffle(buffer_size).repeat(-1).batch(batch_size)
    tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()
    image, label = tfrecord_iterator.get_next()
    return image, label

In [18]:
def file_list_from_folder(folder, data_path):
  folderpath = os.path.join(data_path, folder)
  filelist = []
  for filename in os.listdir(folderpath):
      if filename.startswith('part-') and not filename.endswith('gstmp'):
          filelist.append(os.path.join(folderpath, filename))
  return filelist

In [20]:
file_obj

<_io.BufferedWriter name='testfile'>

In [21]:
filelist = [file_obj]

In [29]:
tf.convert_to_tensor(file_obj)

ValueError: Attempt to convert a value (<_io.BufferedWriter name='testfile'>) with an unsupported type (<class '_io.BufferedWriter'>) to a Tensor.

In [36]:
features = {
  'B1': tf.io.FixedLenFeature([], tf.string),
  'B2': tf.io.FixedLenFeature([], tf.string),
  'B3': tf.io.FixedLenFeature([], tf.string),
  'B4': tf.io.FixedLenFeature([], tf.string),
  'B5': tf.io.FixedLenFeature([], tf.string),
  'B6': tf.io.FixedLenFeature([], tf.string),
  'B7': tf.io.FixedLenFeature([], tf.string),
  'B8': tf.io.FixedLenFeature([], tf.string),
  'B9': tf.io.FixedLenFeature([], tf.string),
  'B10': tf.io.FixedLenFeature([], tf.string),
  'B11': tf.io.FixedLenFeature([], tf.string),
  'label': tf.io.FixedLenFeature([], tf.int64),
}

In [41]:
tf.io.parse_single_example(file_obj,features)

ValueError: Attempt to convert a value (<_io.BufferedWriter name='testfile'>) with an unsupported type (<class '_io.BufferedWriter'>) to a Tensor.

In [12]:
def load_data_gcp():
    client = storage.Client()
    bucket = client.get_bucket('tfrecords_data')
    train = file_list_from_gcp("train", bucket)
    val = file_list_from_gcp("val", bucket)
    return train, val

def file_list_from_gcp(folder, bucket):
    filelist = []
    client = storage.Client()
    for filename in list(client.list_blobs(bucket)):
      if str(filename).startswith("<Blob: tfrecords_data, data/"+folder+'/part-'):
        name = str(filename)
        name = name[:-19]
        name = name.replace('<Blob: tfrecords_data, ', '')
        filelist.append(name)
    file_obj_list = []
    for items in filelist:
      blob = bucket.get_blob(items) 
      blob = bucket.blob(items)
      blobstring = blob.download_as_string()
      with open("testfile", "wb") as file_obj:
          file_obj_list.append(blob.download_to_file(file_obj))
    return file_obj_list


def parse_tfrecords(filelist, batch_size, buffer_size, include_viz=False):
# try a subset of possible bands
    def _parse_(serialized_example, keylist=['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']):
        example = tf.io.parse_single_example(serialized_example, features)

        def getband(example_key):
            img = tf.io.decode_raw(example_key, tf.uint8)
            return tf.reshape(img[:IMG_DIM**2], shape=(IMG_DIM, IMG_DIM, 1))

        bandlist = [getband(example[key]) for key in keylist]
        # combine bands into tensor
        image = tf.concat(bandlist, -1)

        # one-hot encode ground truth labels
        label = tf.cast(example['label'], tf.int32)
        label = tf.one_hot(label, NUM_CLASSES)

        return {'image': image}, label

    tfrecord_dataset = tf.data.TFRecordDataset(filelist)
    tfrecord_dataset = tfrecord_dataset.map(lambda x:_parse_(x)).shuffle(buffer_size).repeat(-1).batch(batch_size)
    tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()
    image, label = tfrecord_iterator.get_next()
    return image, label

    #client = storage.Client()
    train_tfrecords, val_tfrecords = load_data_gcp()
    return train_tfrecords, val_tfrecords

In [ ]:
train, val = load_data_gcp()

In [ ]:
parse_tfrecords(filelist, 100, 100, include_viz=False)

In [3]:
client = storage.Client()

In [11]:
list(client.list_blobs(bucket))[50:53]

[<Blob: tfrecords_data, data/train/part-r-00044, 1606760874449461>,
 <Blob: tfrecords_data, data/train/part-r-00045, 1606835516612750>,
 <Blob: tfrecords_data, data/train/part-r-00046, 1606801041262962>]

In [7]:
bucket = client.get_bucket('tfrecords_data')

In [12]:
def file_list_from_gcp(folder, bucket):
    filelist = []
    for filename in list(client.list_blobs(bucket))[50:53]:
        if str(filename).startswith("<Blob: tfrecords_data, data/"+folder+'/part-'):
            name = str(filename)
            name = name[:-19]
            name = name.replace('<Blob: tfrecords_data, ', '')
            filelist.append(name)
    file_obj_list = []
    for items in filelist:
        blob = bucket.get_blob(items) 
        with open(items, "wb") as file_obj:
            blob.download_to_file(file_obj)
    return filelist

In [16]:
result = file_list_from_gcp('train', bucket)

In [17]:
result

['data/train/part-r-00044',
 'data/train/part-r-00045',
 'data/train/part-r-00046']